# 深度学习计算

## 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0644, -0.0249, -0.1734,  0.0571, -0.0862,  0.1815,  0.2949, -0.1224,
         -0.3933,  0.3950],
        [-0.0265, -0.0301, -0.0605,  0.0264, -0.1022,  0.1807,  0.3463, -0.1494,
         -0.3957,  0.2957]], grad_fn=<AddmmBackward0>)

### 5.1.1 自定义块

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.1790, -0.0514, -0.3164,  0.1152, -0.2710,  0.1174,  0.2706,  0.3973,
         -0.0314, -0.2181],
        [-0.1663, -0.0056, -0.2658,  0.0817, -0.2799, -0.0422,  0.1720,  0.3296,
         -0.0877, -0.2466]], grad_fn=<AddmmBackward0>)

### 5.1.2 顺序块

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0673, -0.0950, -0.1527, -0.0872, -0.1821,  0.1228,  0.1330,  0.1572,
         -0.0047,  0.0226],
        [-0.0865, -0.1121, -0.0505, -0.0602, -0.1268,  0.1791,  0.2612,  0.0139,
          0.0474,  0.0615]], grad_fn=<AddmmBackward0>)

### 5.1.3 在前向传播函数中执行代码

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)
        
    def forward(self, X):
        X = self.linear(X)
        
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0080, grad_fn=<SumBackward0>)

In [15]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), 
                                 nn.ReLU(),
                                 nn.Linear(64, 32),
                                 nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))

In [16]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0087, grad_fn=<SumBackward0>)

## 5.2 参数管理

In [17]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.3397],
        [0.3857]], grad_fn=<AddmmBackward0>)

### 5.2.1 参数访问

In [18]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1379, -0.0518,  0.2766, -0.3029,  0.2095,  0.1071, -0.1423, -0.3360]])), ('bias', tensor([0.3085]))])


In [19]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

    目标参数

In [20]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3085], requires_grad=True)
tensor([0.3085])


In [21]:
net[2].weight.grad == None

True

    一次性访问所有参数 

In [22]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [23]:
net.state_dict()['2.bias'].data

tensor([0.3085])

    从嵌套块收集参数

In [24]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0939],
        [0.0940]], grad_fn=<AddmmBackward0>)

In [25]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [26]:
rgnet[0][1][0].bias.data

tensor([ 0.0491, -0.4887,  0.3540,  0.4009, -0.0542, -0.1677, -0.2563, -0.2888])

### 5.2.2 参数初始化

    内置初始化

In [27]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0050,  0.0029, -0.0029, -0.0191]), tensor(0.))

In [28]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [29]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [31]:
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.2046,  0.5611, -0.3121, -0.4098])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


    自定义初始化

In [34]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.9752, -7.2364,  0.0000, -0.0000],
        [ 9.6046,  8.2705,  7.6951, -9.2077]], grad_fn=<SliceBackward0>)

### 5.2.3 参数绑定

In [38]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
print(net[1].weight.data[0] == net[3].weight.data[0])
net[1].weight.data[0, 0] = 100
# 确保它们实际上是同⼀个对象，⽽不只是有相同的值
print(net[1].weight.data[0] == net[3].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 5.4 自定义层

### 5.4.1 不带参数的层

In [40]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()

In [42]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [43]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [44]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-2.7940e-09, grad_fn=<MeanBackward0>)

### 5.4.2 带参数的层

In [46]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
   
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [47]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.4126,  0.3029, -0.9116],
        [ 1.1895,  1.1129, -0.7598],
        [ 0.7644, -0.5646,  0.7214],
        [ 0.9314, -1.2136,  1.3467],
        [-1.2148,  1.3960, -0.7739]], requires_grad=True)

In [48]:
linear(torch.rand(2, 5))

tensor([[1.5147, 1.1357, 1.1298],
        [0.2881, 0.9951, 1.2699]])

In [49]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[10.7069],
        [15.0481]])

## 5.5 读写文件

### 5.5.1 加载和保存张量

In [50]:
import torch 
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [51]:
x2 = torch.load('x-file')

In [52]:
x2

tensor([0, 1, 2, 3])

In [53]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [54]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.5.2 加载和保存模型参数

In [55]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [56]:
torch.save(net.state_dict(), 'mlp.params')

In [57]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [58]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 5.6 GPU

### 5.6.1 计算设备

In [60]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [61]:
torch.cuda.device_count()

2

In [63]:
def try_gpu(i=0): #@save
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): #@save
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

### 5.6.2 张量与GPU

In [64]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [65]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [66]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.3959, 0.7125, 0.0651],
        [0.7090, 0.9593, 0.3706]], device='cuda:1')

In [67]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


In [68]:
Y + Z

tensor([[1.3959, 1.7125, 1.0651],
        [1.7090, 1.9593, 1.3706]], device='cuda:1')

In [69]:
Z.cuda(1) is Z

True

### 5.6.3 神经网络与GPU

In [70]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [71]:
net(X)

tensor([[-0.7129],
        [-0.7129]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [72]:
net[0].weight.data.device

device(type='cuda', index=0)